In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
spark = SparkSession.builder.appName('minTemp').getOrCreate()

In [7]:
schema = StructType([StructField("stationID", StringType(), True),
                     StructField("date", IntegerType(), True),
                     StructField("measure_type", StringType(), True),
                     StructField("temperature", FloatType(), True)])

In [8]:
df = spark.read.schema(schema).csv('data/1800.csv')

In [9]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [10]:
minTemp = df.filter(df.measure_type == "TMIN")

In [11]:
minTemp.show()

+-----------+--------+------------+-----------+
|  stationID|    date|measure_type|temperature|
+-----------+--------+------------+-----------+
|ITE00100554|18000101|        TMIN|     -148.0|
|EZE00100082|18000101|        TMIN|     -135.0|
|ITE00100554|18000102|        TMIN|     -125.0|
|EZE00100082|18000102|        TMIN|     -130.0|
|ITE00100554|18000103|        TMIN|      -46.0|
|EZE00100082|18000103|        TMIN|      -73.0|
|ITE00100554|18000104|        TMIN|      -13.0|
|EZE00100082|18000104|        TMIN|      -74.0|
|ITE00100554|18000105|        TMIN|       -6.0|
|EZE00100082|18000105|        TMIN|      -58.0|
|ITE00100554|18000106|        TMIN|       13.0|
|EZE00100082|18000106|        TMIN|      -57.0|
|ITE00100554|18000107|        TMIN|       10.0|
|EZE00100082|18000107|        TMIN|      -50.0|
|ITE00100554|18000108|        TMIN|       14.0|
|EZE00100082|18000108|        TMIN|      -31.0|
|ITE00100554|18000109|        TMIN|       23.0|
|EZE00100082|18000109|        TMIN|     

In [12]:
stationTemp = minTemp.select("stationID", 'temperature')

In [14]:
minTempsByStation = stationTemp.groupBy('stationID').min('temperature')

In [15]:
minTempsByStation.show()

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [26]:
minTempsByStationF = minTempsByStation.withColumn("temperature",
                                                  func.round(func.col("min(temperature)") * 0.1 * (9.0 / 5.0) + 32.0, 2))\
                                                  .select("stationID", "temperature").sort("temperature", ascending = True)

In [24]:
results = minTempsByStationF.collect()

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

EZE00100082	7.70F
ITE00100554	5.36F
